ehleed linkuudeee awna awsan link burees acttive tab 11 bnu shalgaad baiwal mongol angli awaaad baihgui bol mongol awaad 

In [6]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Setup Chrome with Selenium using Service
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

num_pages = 45
base_url = 'https://legalinfo.mn/mn/law?page=law&cate=27&active=1&page=1&sort=title&page='

all_links = []

for page_num in range(1, num_pages+1):
    url = base_url + str(page_num)
    driver.get(url)
    wait = WebDriverWait(driver, 20) 

    try:
        # Wait for the 'shine-huuli-main' div to be loaded
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.shine-huuli-main"))
        )
        time.sleep(5)  # Wait for 5 seconds before loading the next page
        # Extract information
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')

        # Find the container with the specified class
        container = soup.find("div", {"class": "shine-huuli-main"})
        if container:
            # Extract links that have the specific class "act-name"
            links = [a['href'] for a in container.select('a.act-name')]
            all_links.extend(links)
            print(f"Links from Page {page_num}:", links)
        else:
            print("No container with the specified class found.")

    finally:
        # Ensure the browser is closed even if an error occurs
        print("End of Page", page_num)
        time.sleep(1)  # Wait for 5 seconds before loading the next page

print("All Links:", all_links)
driver.quit()


Links from Page 1: []
End of Page 1
End of Page 2


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=124.0.6367.61)
Stacktrace:
	GetHandleVerifier [0x00CCC113+48259]
	(No symbol) [0x00C5CA41]
	(No symbol) [0x00B50A17]
	(No symbol) [0x00B2E02B]
	(No symbol) [0x00BB742E]
	(No symbol) [0x00BC9476]
	(No symbol) [0x00BB0B36]
	(No symbol) [0x00B8570D]
	(No symbol) [0x00B862CD]
	GetHandleVerifier [0x00F86533+2908323]
	GetHandleVerifier [0x00FC3B4B+3159739]
	GetHandleVerifier [0x00D6505B+674763]
	GetHandleVerifier [0x00D6B21C+699788]
	(No symbol) [0x00C66244]
	(No symbol) [0x00C62298]
	(No symbol) [0x00C6242C]
	(No symbol) [0x00C54BB0]
	BaseThreadInitThunk [0x767CFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77C67C5E+286]
	RtlGetAppContainerNamedObjectPath [0x77C67C2E+238]


In [1]:
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import json
import re

def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver



def fetch_texts(url):
    driver = setup_driver()
    texts_mon = ""
    texts_eng = ""
    title = ""

    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract title using the new method
        title_tag = soup.find('p', style=lambda value: value and 'padding-left:' in value)
        title = title_tag.get_text(strip=True) if title_tag else "Title not found"

        # Fetch Mongolian text
        element_mon = soup.find('div', class_='maincontenter w-100 pull-left')
        if element_mon:
            texts_mon = element_mon.get_text(strip=True).replace("Хэвлэх", "")

        # Navigate to the page with Selenium to check for the active tab
        driver.get(url)
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'a[href="#active-tab-11"]')))
            # Fetch English text
            element_eng = soup.find('div', class_='w-100 pull-left --nomigration')
            if element_eng:
                texts_eng = element_eng.get_text(strip=True)
        except:
            # Handle cases where the active tab 11 does not exist
            texts_eng = "not."

    except Exception as e:
        print(f"Error processing {url}: {str(e)}")
    finally:
        driver.quit()

    return {'url': url, 'title': title, 'mon': texts_mon, 'eng': texts_eng}

# List of URLs to process

# Use ThreadPoolExecutor to handle multiple URLs in parallel
with ThreadPoolExecutor(max_workers=5) as executor:
    results = list(executor.map(fetch_texts, all_links))

# Write results to JSONL file
with open('titleless1.jsonl', 'w', encoding='utf-8') as f:
    for result in results:
        json_line = json.dumps({"title": result['title'], "mon": result['mon'], "eng": result['eng']}, ensure_ascii=False)
        f.write(json_line + '\n')

print("Processing complete. Data written to output.jsonl.")


NameError: name 'all_links' is not defined